# Video Donwload
This script is used to download videos from kinects dataset.
___
Has a python class with a class helper builder from this in classpath.

## Download the zip
Make a download resources and save it in dataset path.

In [1]:
%%bash
mkdir dataset
curl https://storage.googleapis.com/deepmind-media/research/Kinetics_700.zip --output dataset/kinects_700.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 21.7M  100 21.7M    0     0  13.6M      0  0:00:01  0:00:01 --:--:-- 13.6M


extract the infos from zip file

In [2]:
%%bash
cd dataset
unzip kinects_700.zip

Archive:  kinects_700.zip
  inflating: kinetics_700_readme.txt  
   creating: __MACOSX/
  inflating: __MACOSX/._kinetics_700_readme.txt  
  inflating: kinetics_700_train.zip  
  inflating: __MACOSX/._kinetics_700_train.zip  
  inflating: kinetics_700_test.zip   
  inflating: __MACOSX/._kinetics_700_test.zip  
  inflating: kinetics_700_val.zip    
  inflating: __MACOSX/._kinetics_700_val.zip  


In [3]:
%%bash
cd dataset
unzip kinetics_700_train.zip
unzip kinetics_700_test.zip
unzip kinetics_700_val.zip

Archive:  kinetics_700_train.zip
  inflating: kinetics_700_train.csv  
  inflating: kinetics_700_train.json  
Archive:  kinetics_700_test.zip
  inflating: kinetics_700_test.csv   
  inflating: kinetics_700_test.json  
Archive:  kinetics_700_val.zip
  inflating: kinetics_700_val.csv    
  inflating: kinetics_700_val.json   


## Create variables
Create variables with const values

In [1]:
import os

In [2]:
def create_file_structure(path, folders_names):
    '''
    this method create a folders in path
    '''
    mapping = {}
    if not os.path.exists(path):
        os.mkdir(path)
    for name in folders_names:
        dir_ = os.path.join(path, name)
        if not os.path.exists(dir_):
            os.mkdir(dir_)
        mapping[name] = dir_
    return mapping

In [3]:
REQUIRED_COLUMNS = ['label', 'youtube_id', 'time_start', 'time_end', 'split', 'is_cc']
TRIM_FORMAT = '%06d'
URL_BASE = 'https://www.youtube.com/watch?v='

VIDEO_EXTENSION = '.mp4'
VIDEO_FORMAT = 'mp4'

In [4]:
TRAIN_FOLDER = 'train'
VALIDATE_FOLDER = 'validate'
TEST_FOLDER = 'test'

In [5]:
BASE_PATH = os.path.join('/', 'home', os.environ['USER'], 'kinects')

TRAIN_VIDEOS_PATH = os.path.join(BASE_PATH, TRAIN_FOLDER)
VALIDATE_VIDEOS_PATH = os.path.join(BASE_PATH, VALIDATE_FOLDER)
TEST_VIDEOS_PATH = os.path.join(BASE_PATH, TEST_FOLDER)

### Create files
this files is used for save the videos after download

In [6]:
if not os.path.exists(BASE_PATH):
    os.mkdir(BASE_PATH)
else:
    print(f'Path {BASE_PATH} already exists!')

Path /home/renato/kinects already exists!


In [7]:
# file to save train, validate and test
create_file_structure(BASE_PATH, [TRAIN_FOLDER, VALIDATE_FOLDER, TEST_FOLDER])

{'test': '/home/renato/kinects/test',
 'train': '/home/renato/kinects/train',
 'validate': '/home/renato/kinects/validate'}

## Video Trimming
That method is used to trimming videos before download

In [8]:
import subprocess
import ffmpeg

In [9]:
def trim(row, label_to_dir, test=False):
    label = row['label'] if not test else ''
    filename = row['youtube_id']
    time_start = row['time_start']
    time_end = row['time_end']

    input_filename = os.path.join(label_to_dir['tmp'], f'{filename}{VIDEO_EXTENSION}')
    output_filename = os.path.join(label_to_dir[label], f'{filename}{VIDEO_EXTENSION}')

    if os.path.exists(output_filename):
        print('Already trimmed: ', filename)
    else:
        print('Start trimming: ', filename)

        try:
            ffmpeg.trim(ffmpeg.input(input_filename), start=time_start, end=time_end).output(
                output_filename).run()
        except Exception as e:
            print(f'Error in trimming: {e}')

        print('Finish trimming: ', filename)

## Method to video download
Make a download of a video

In [10]:
import pandas as pd
import pytube

In [11]:
def download_clip(row, label_to_dir, test=False):
    filename = row['youtube_id']

    if not os.path.exists(os.path.join(label_to_dir['tmp'], filename + VIDEO_EXTENSION)):
        print('Start downloading: ', filename)
        try:
            pytube.YouTube(URL_BASE + filename) \
                .streams \
                .filter(subtype=VIDEO_FORMAT) \
                .first() \
                .download(label_to_dir['tmp'], filename)
            print('Finish downloading: ', filename)
        except KeyError as e:
            print(f'Key Error {e}')
            return
        except Exception as e:
            print(f'Error in download video: {e}')
            return
    else:
        print('Already downloaded: ', filename)

## Donwload
Create a method to make a download and trim the videos. Repair that I delete the `tmp` file.

In [12]:
import shutil

In [13]:
def download(path_csv, target, heads=5, test=False):
    links_data_frames = pd.read_csv(path_csv).head(heads)

    if not test:
        folders_names = links_data_frames['label'].unique().tolist() + ['tmp']
        label_to_dir = create_file_structure(path=target, folders_names=folders_names)

        [download_clip(row, label_to_dir) for _, row in links_data_frames.iterrows()]
        [trim(row, label_to_dir) for _, row in links_data_frames.iterrows()]

        shutil.rmtree(label_to_dir['tmp'])
    else:
        folders_names = ['tmp', '']
        label_to_dir = create_file_structure(path=target, folders_names=folders_names)

        [download_clip(row, label_to_dir) for _, row in links_data_frames.iterrows()]
        [trim(row, label_to_dir, test=True) for _, row in links_data_frames.iterrows()]

        shutil.rmtree(label_to_dir['tmp'])

### Download of Train test
Make a test download of a five videos from train
___

The videos path is `dataset/kinetics_700_train.csv` and the target path is the variable `TRAIN_VIDEOS_PATH`.

In [14]:
download('dataset/kinetics_700_train.csv', TRAIN_VIDEOS_PATH)

Start downloading:  oJCxnjaCoyI
Finish downloading:  oJCxnjaCoyI
Start downloading:  XtbeP16BJdQ
Finish downloading:  XtbeP16BJdQ
Start downloading:  XXulvmGzw60
Finish downloading:  XXulvmGzw60
Start downloading:  -3B32lodo2M
Error in download video: HTTP Error 403: Forbidden
Start downloading:  -7kbO0v4hag
Finish downloading:  -7kbO0v4hag
Start trimming:  oJCxnjaCoyI
Finish trimming:  oJCxnjaCoyI
Start trimming:  XtbeP16BJdQ
Finish trimming:  XtbeP16BJdQ
Start trimming:  XXulvmGzw60
Finish trimming:  XXulvmGzw60
Start trimming:  -3B32lodo2M
Error in trimming: ffmpeg error (see stderr output for detail)
Finish trimming:  -3B32lodo2M
Start trimming:  -7kbO0v4hag
Finish trimming:  -7kbO0v4hag


### Download of Validate test
Make a test download of a five videos from train
___

The videos path is `dataset/kinetics_700_val.csv` and the target path is the variable `VALIDATE_VIDEOS_PATH`.

In [15]:
download('dataset/kinetics_700_val.csv', VALIDATE_VIDEOS_PATH)

Start downloading:  ixq5OGYjjmA
Finish downloading:  ixq5OGYjjmA
Start downloading:  j2_AKSjeQy0
Error in download video: HTTP Error 403: Forbidden
Start downloading:  J_faWIyCzdc
Finish downloading:  J_faWIyCzdc
Start downloading:  Kbmv94BqZzU
Finish downloading:  Kbmv94BqZzU
Start downloading:  LFMlCcsLdKA
Error in download video: HTTP Error 403: Forbidden
Start trimming:  ixq5OGYjjmA
Finish trimming:  ixq5OGYjjmA
Start trimming:  j2_AKSjeQy0
Error in trimming: ffmpeg error (see stderr output for detail)
Finish trimming:  j2_AKSjeQy0
Start trimming:  J_faWIyCzdc
Finish trimming:  J_faWIyCzdc
Start trimming:  Kbmv94BqZzU
Finish trimming:  Kbmv94BqZzU
Start trimming:  LFMlCcsLdKA
Error in trimming: ffmpeg error (see stderr output for detail)
Finish trimming:  LFMlCcsLdKA


### Download of Test
Make a test download of a five videos from test
___

The videos path is `dataset/kinetics_700_test.csv` and the target path is the variable `TEST_VIDEOS_PATH`.

In [17]:
download('dataset/kinetics_700_test.csv', TEST_VIDEOS_PATH, heads=8, test=True)

Start downloading:  6dEpI75FOeo
Finish downloading:  6dEpI75FOeo
Start downloading:  15H3EqaHVi0
Error in download video: HTTP Error 403: Forbidden
Start downloading:  d6Ko4hm8M8E
Finish downloading:  d6Ko4hm8M8E
Start downloading:  -SakeFNtM0s
Finish downloading:  -SakeFNtM0s
Start downloading:  at3mGS-FAVg
Finish downloading:  at3mGS-FAVg
Start downloading:  0-qpI81QREc
Finish downloading:  0-qpI81QREc
Start downloading:  G5naKwYvSFM
Finish downloading:  G5naKwYvSFM
Start downloading:  JlaQtvL6tgQ
Finish downloading:  JlaQtvL6tgQ
Start trimming:  6dEpI75FOeo
Finish trimming:  6dEpI75FOeo
Start trimming:  15H3EqaHVi0
Error in trimming: ffmpeg error (see stderr output for detail)
Finish trimming:  15H3EqaHVi0
Start trimming:  d6Ko4hm8M8E
Finish trimming:  d6Ko4hm8M8E
Start trimming:  -SakeFNtM0s
Finish trimming:  -SakeFNtM0s
Start trimming:  at3mGS-FAVg
Finish trimming:  at3mGS-FAVg
Start trimming:  0-qpI81QREc
Finish trimming:  0-qpI81QREc
Start trimming:  G5naKwYvSFM
Finish trimming: